<a href="https://colab.research.google.com/github/nojihun/save/blob/main/AI_05_%EB%85%B8%EC%A7%80%ED%9B%88_section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
from google.colab import files
import pandas as pd
from scipy import stats
uploaded = files.upload()

Saving vgames2.csv to vgames2.csv


In [69]:
df = pd.read_csv('vgames2.csv')
df = df.dropna() #결측치 제거

지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.</p>
연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.</p>
출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.

In [70]:
def my_1(x):# 매출액서 K 와 M을 빼내기 위한 함수
  val= x.find('K')
  val2=x.find('M')
  if val != -1:
    x = x.replace('K','')
    x = float(x)
    x = x*1000
  elif val2 != - 1:
    x=x.replace('M','')
    x=float(x)
    x = x*1000000
  else:
    x = float(x)
  return x

df['NA_Sales']= df['NA_Sales'].apply(my_1)
df['EU_Sales']= df['EU_Sales'].apply(my_1)
df['JP_Sales']= df['JP_Sales'].apply(my_1)
df['Other_Sales']= df['Other_Sales'].apply(my_1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   16241 non-null  int64  
 1   Name         16241 non-null  object 
 2   Platform     16241 non-null  object 
 3   Year         16241 non-null  float64
 4   Genre        16241 non-null  object 
 5   Publisher    16241 non-null  object 
 6   NA_Sales     16241 non-null  float64
 7   EU_Sales     16241 non-null  float64
 8   JP_Sales     16241 non-null  float64
 9   Other_Sales  16241 non-null  float64
dtypes: float64(5), int64(1), object(4)
memory usage: 1.4+ MB


In [71]:
df.head(12)

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
5,6,Power Pro Kun Pocket 8,DS,2005.0,Sports,Konami Digital Entertainment,0.00,0.00,0.14,0.00
6,7,JASF: Jane's Advanced Strike Fighters,PC,2011.0,Simulation,Deep Silver,0.02,0.01,0.00,0.00
7,8,Samurai Warriors: State of War,PSP,2005.0,Action,Tecmo Koei,0.07,0.00,0.11,0.01
8,9,Imabikisou,PS3,2007.0,Adventure,Sega,0.00,0.00,0.04,0.00
9,10,The Penguins of Madagascar: Dr. Blowhole Retur...,PS3,2011.0,Action,THQ,0.04,0.03,0.00,0.01


In [76]:
a = [] # 연도에 오류가나서 제대로 표시 되지 않은 데이터들을 제거
for i in df['Year']:
  if i < 1900 :
      a.append(i)

de = df[df['Year']<=1899].index
df = df.drop(de)


In [78]:
df=df.drop(['Unnamed: 0'], axis ='columns' )
df.head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0.00,0.00,0.00
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0.00,0.00,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0.00,0.00,0.02,0.00
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0.00,0.00,0.00
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0.00,0.04
